In [48]:
import numpy as np
import matplotlib.pyplot as plt

In [49]:
class HebianNet():
    def __init__(self):
        self.weights = None
        self.trained = False
        
    def train(self,data):
        if not self.trained:
            self.weights = np.zeros((data.shape[1], data.shape[1]))
        for index in range(data.shape[0]):
            self.weights = np.add(np.outer(np.transpose(data[index,:]),data[index,:]),self.weights)
        self.trained = True
        
    def train_asynchronous():
        ###Memes
        return
    
    def update(self, x):
        new = np.dot(x,self.weights)
        new[new>=0] = 1
        new[new<0] = -1
        return new
    
    def converge(self, x):
        error = 1
        itercount = 0
        while error > 0:
            new = self.update(x)
            error = sum(abs(new-x))
            x = new.copy()
            itercount += 1
        return x, itercount
    
    def energy(self, pattern):
        outer = np.outer(pattern, np.transpose(pattern))
        

### Hopfield network recall

In [50]:
x1 = np.array([-1,-1,1,-1,1,-1,-1,1])
x2 = np.array([-1,-1,-1,-1,-1,1,-1,-1])
x3 = np.array([-1,1,1,-1,-1,1,-1,1])
data  = np.array([x1,x2,x3])

In [51]:
hn = HebianNet()

In [52]:
hn.train(data)

Reproduces learned patterns

In [53]:
hn.update(x1)-x1

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

### Convergence and attractors

In [54]:
x1d = np.array([1,-1,1,-1,1,-1,-1,1])
x2d = np.array([1,1,-1,-1,-1,1,-1,-1])
x3d = np.array([1,1,1,-1,1,1,-1,1])

In [55]:
hn.converge(x1d)[0]- x1

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [56]:
hn.converge(x2d)[0] - x2

array([ 0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])

In [57]:
hn.converge(x3d)[0] - x3

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

Only x1d and x3d converge towards their respective undistorted patterns. x2d does not.

In [58]:
searchgrid = np.ones((256,8))

In [59]:
searchgrid

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

#### More than half wrong

In [60]:
x1dd = np.multiply(x1,np.array([-1,-1,-1,-1,-1,1,1,1]))

In [61]:
new, iterations = hn.converge(x1dd)
new = np.multiply(new,x1).copy()
result = len(new[new<0])
print("Diff from original vector in {} elements. Number of iterations to convergance was {}".format(result,iterations))

Diff from original vector in 4 elements. Number of iterations to convergance was 3


In [62]:
hn.converge(x1dd)

(array([ 1.,  1.,  1.,  1.,  1., -1.,  1.,  1.]), 3)

In [63]:
x1

array([-1, -1,  1, -1,  1, -1, -1,  1])

In [64]:
"CHECK 1 2 3"

'CHECK 1 2 3'

### Sequential Update

In [65]:
data = np.loadtxt('./pict.dat',delimiter=",", dtype=int).reshape(-1,1024)

In [66]:
trainingData = data[0:3].copy()

In [67]:
testData = data[3:].copy()

In [68]:
hn = HebianNet()

In [69]:
hn.train(trainingData)

#### Check stability of training patterns

In [75]:
test1 = np.multiply(hn.update(trainingData[0]),trainingData[0])
print(len(test1[test1<0]))

0


In [76]:
test2 = np.multiply(hn.update(trainingData[1]),trainingData[1])
print(len(test1[test1<0]))

0


In [77]:
test3 = np.multiply(hn.update(trainingData[2]),trainingData[2])
print(len(test1[test1<0]))

0


#### Check performance on degraded patterns

In [99]:
test1 = hn.converge(testData[-1])[0]
test2 = np.multiply(test1,trainingData[2])
print(len(test2[test2<0]))
### p11 (a combination of p2 and p3) comes closest to p3 

121


In [100]:
test3 = hn.converge(testData[-2])[0]
test4 = np.multiply(test3,trainingData[0])
print(len(test4[test4<0]))
### p10 (a degraded version of p1) converges to p1

0
